In [1]:
from dotenv import load_dotenv, dotenv_values

load_dotenv("/opt/app-root/src/hazard/credentials-dev.env", override=True)
import logging, sys

sys.path.append(r"/opt/app-root/src/hazard/src")
from hazard.models.drought_index import DroughtIndicator, S3ZarrWorkingStore, BatchItem, ZarrWorkingStore, ProgressStore
import os
import s3fs
import numpy as np
import xarray as xr
from pathlib import PurePosixPath
from hazard.sources.osc_zarr import OscZarr

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s",
    handlers=[logging.FileHandler(filename="batch.log"), logging.StreamHandler(sys.stdout)],
)

## Model Run

In [ ]:
S3Zarr = S3ZarrWorkingStore()
target = OscZarr()
model = DroughtIndicator(S3Zarr)
item = BatchItem(gcm=model.gcms[5], scenario=model.scenarios[3], central_years=model.central_years)
progress = ProgressStore(dir="/opt/app-root/src/hazard", id=item.gcm + "_" + item.scenario)
print(vars(item))
model.pre_chunk(item)
model.run_single(item, target=target, progress_store=progress)

{'gcm': 'ACCESS-CM2', 'scenario': 'ssp585', 'central_years': [2005, 2030, 2040, 2050, 2080]}
[2023-12-12 18:51:38,430] {drought_index.py:274} INFO - calculated SPEI for gcm=ACCESS-CM2, lats=[(10.0, 20.0)], lons=[(300.0, 310.0)]
[2023-12-12 18:51:39,214] {drought_index.py:240} INFO - written chunk Chunk_0282 to zarr array.
[2023-12-12 18:51:39,218] {drought_index.py:206} INFO - chunk Chunk_0282 complete.


In [2]:
path = model.resource.path.format(gcm=item.gcm, scenario=item.scenario, year=2050)
months_spei12m_below_set = target.read(path)
months_spei12m_below_set.sel(index=0).plot(figsize=[20, 10])

NameError: name 'ProgressStore' is not defined